<h2> 3.6 Featurizing text data with tfidf weighted word-vectors </h2>

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time

import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
#import spacy

In [24]:

df = pd.read_csv("Desktop/trainquestion.csv")
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

In [25]:
del df['Unnamed: 0']
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,qid1fre,qid2fre,q1len,q2len,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,1,1,66,57,...,0.785709,0.0,1.0,2.0,13.0,100,93,93,100,0.982759
1,1,3,4,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...,0,1,1,51,88,...,0.466664,0.0,1.0,5.0,12.5,86,63,66,75,0.596154
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,1,1,73,59,...,0.285712,0.0,1.0,4.0,12.0,66,66,54,54,0.166667
3,3,7,8,why am i mentally very lonely how can i solve...,find the remainder when math 23 24 math i...,0,1,1,50,65,...,0.000000,0.0,0.0,2.0,12.0,36,36,35,40,0.039216
4,4,9,10,which one dissolve in water quikly sugar salt...,which fish would survive in salt water,0,1,1,76,39,...,0.307690,0.0,1.0,6.0,10.0,67,47,46,56,0.175000


In [26]:
y=df['is_duplicate']
X=df.drop('is_duplicate',axis=1)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)

In [28]:
dfx=X_test

In [42]:
y_test.to_csv('Desktop/ytesting.csv')

C:\Users\vivek\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [43]:
y_train.to_csv('Desktop/ytraining.csv')

C:\Users\vivek\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [29]:

# merge texts
questions = list(dfx['question1']) + list(dfx['question2'])

tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

- After we find TF-IDF scores, we convert each question to a weighted average of word2vec vectors by these scores.

In [30]:
import spacy

In [31]:

nlp = spacy.load('en_core_web_sm')

vecs1 = []
for qu1 in tqdm(list(dfx['question1'])):
    doc1 = nlp(qu1) 

    mean_vec1 = np.zeros([len(doc1), 384])
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
dfx['q1_feats_m'] = list(vecs1)


100%|██████████████████████████████████████████████████████████████████████████| 133436/133436 [22:29<00:00, 98.90it/s]
C:\Users\vivek\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [19]:
del dfx['q3_feats_m']

In [33]:

vecs1 = []
for qu1 in tqdm(list(dfx['question2'])):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), 384])
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
dfx['q2_feats_m'] = list(vecs1)

100%|██████████████████████████████████████████████████████████████████████████| 133436/133436 [23:16<00:00, 95.55it/s]
C:\Users\vivek\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# Check for the vectorisation and weather the comlums are good or empty. 

In [34]:
y=0
for x in tqdm(list(dfx['q2_feats_m'])):
    for a in x:
        if(a==0):
            continue
        else:
            y=1
            break
    if(y==1):
        print('No')
        break
if(y==0):
    print('everything is 0')

  0%|                                                                                       | 0/133436 [00:00<?, ?it/s]

No


In [35]:
dfx

,id,qid1,qid2,question1,question2,qid1fre,qid2fre,q1len,q2len,q1words,...,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio,q1_feats_m,q2_feats_m
120567,120567,238932,238933,how does the boggart work,what would the boggart of a boggart be,1,1,26,39,5,...,0.0,3.0,6.5,69,57,57,69,0.481481,"[24.400227546691895, 24.29525923728943, 32.268...","[45.22122001647949, 19.814120322465897, 37.314..."
324466,324466,636476,636477,what is difference between project manager and...,what are the differences between project manag...,1,1,63,76,9,...,1.0,1.0,9.5,76,74,76,76,0.359375,"[62.285337686538696, 32.274497866630554, 7.508...","[69.63745212554932, 32.16139742732048, 14.8635..."
398558,398558,778728,778729,what hotel in jabalpur would be safe for unmar...,what hotel in allahabad would be safe for unma...,1,1,124,125,20,...,1.0,0.0,20.0,96,92,96,95,0.816000,"[40.846871703863144, 126.04081556200981, 40.31...","[39.950090289115906, 120.48273184895515, 28.32..."
339914,339914,666314,666315,what is stronger super saiyan 4 or super sai...,how does gohan turn into super saiyan 2,1,1,54,40,11,...,0.0,2.0,9.0,60,53,59,60,0.341463,"[24.22315703332424, 14.586243331432343, 27.425...","[30.55031893402338, -23.009349584579468, 7.954..."
185732,185732,366764,366765,how do i fill in address line 1 and address li...,how do i register desired web address,1,1,51,38,12,...,1.0,5.0,9.5,62,60,57,55,0.230769,"[75.55021311342716, 43.31147062778473, 63.7920...","[85.07596880197525, 62.78585124015808, 48.8965..."
383446,383446,749820,749821,what is the procedure to join the bomb detecti...,where can we get medical reports in delhi,1,1,97,42,15,...,0.0,7.0,11.5,41,39,38,43,0.069767,"[117.36789935827255, 100.25205159187317, 25.96...","[7.689534068107605, -12.567209959030151, 26.11..."
76919,76919,152835,152836,is google actually replacing nexus with pixel,why is google replacing nexus with pixel,1,1,46,41,7,...,0.0,0.0,7.0,95,85,85,78,0.666667,"[-33.896195471286774, 17.2117258310318, 33.005...","[-8.482747554779053, 32.01514720916748, 64.852..."
62080,62080,123460,123461,why did dumbledore not throw away the potion i...,why can not dumbledore just scoop and throw aw...,1,1,98,93,18,...,1.0,1.0,17.5,65,58,54,55,0.163265,"[80.41992253065109, 8.98958496004343, 29.45121...","[2.5115463733673096, 84.09768778085709, -6.257..."
227108,227108,447589,447590,if the arab societies believe the 1919 1920 bo...,do the arab societies believe the 1919 1929 bo...,1,1,146,89,24,...,0.0,10.0,20.0,97,74,72,97,0.511111,"[-45.328601226210594, 143.27423176169395, 199....","[30.2679410725832, 126.18296406418085, 122.392..."
393826,393826,769659,769660,what are some manufacturing organizations,what is manufacturing,1,1,42,22,5,...,1.0,2.0,4.0,92,61,65,77,0.652174,"[-7.114170551300049, 29.11326026916504, 24.345...","[-19.79277729988098, -5.950626254081726, 18.07..."


In [5]:
import pickle

In [36]:
pickle_out=open('vectortesting','wb')
pickle.dump(dfx,pickle_out)

In [37]:
pickle_out.close()

In [6]:
pickll=open('vector','rb')
dfx=pickle.load(pickll)

In [38]:
#prepro_features_train.csv (Simple Preprocessing Feartures)
#nlp_features_train.csv (NLP Features)
if os.path.isfile('Desktop/trainquestion.csv'):
    dfnlp = pd.read_csv("Desktop/trainquestion.csv",encoding='utf-8')
else:
    print("download nlp_features_train.csv from drive or run previous notebook")

if os.path.isfile('Desktop/questions.csv'):
    dfppro = pd.read_csv("Desktop/questions.csv",encoding='utf-8')
else:
    print("download df_fe_without_preprocessing_train.csv from drive or run previous notebook")

In [39]:
df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3 = dfx.drop(['qid1','qid2','question1','question2'],axis=1)
df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)
df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)

In [11]:
# dataframe of nlp features
df1.head()

,Unnamed: 0,id,is_duplicate,qid1fre,qid2fre,q1len,q2len,q1words,q2words,wordcommon,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,0,0,1,1,66,57,14,12,10.0,...,0.785709,0.0,1.0,2.0,13.0,100,93,93,100,0.982759
1,1,1,0,1,1,51,88,8,13,4.0,...,0.466664,0.0,1.0,5.0,12.5,86,63,66,75,0.596154
2,2,2,0,1,1,73,59,14,10,4.0,...,0.285712,0.0,1.0,4.0,12.0,66,66,54,54,0.166667
3,3,3,0,1,1,50,65,11,9,0.0,...,0.000000,0.0,0.0,2.0,12.0,36,36,35,40,0.039216
4,4,4,0,1,1,76,39,13,7,2.0,...,0.307690,0.0,1.0,6.0,10.0,67,47,46,56,0.175000


In [12]:
# data before preprocessing 
df2.head()

,id
0,0
1,1
2,2
3,3
4,4


In [17]:
# Questions 1 tfidf weighted word2vec
df3_q1.head()

,0,1,2,3,4,5,6,7,8,9,...,375,376,377,378,379,380,381,382,383,id
181139,-4.806136,12.040295,34.272986,15.230215,57.987459,-49.585320,-73.533863,-43.069118,-2.229854,32.080197,...,19.839034,6.810962,-4.031752,1.051718,0.319788,-17.768475,7.925187,-3.628736,-1.526461,181139
284223,164.180476,34.077796,48.238049,139.394157,48.564777,24.024287,-177.166243,-46.163154,29.838822,128.012886,...,14.548875,13.339853,-8.493817,5.232039,-4.578188,-11.205049,-12.062581,14.141603,4.723512,284223
314840,73.169844,21.476749,30.279141,84.283732,-60.349084,-105.217798,-408.981576,-86.154419,139.074083,75.015152,...,58.979260,-2.050193,-7.778956,26.539724,9.355025,-24.832465,7.982635,19.100337,5.061825,314840
337099,6.383046,33.568555,37.774491,-2.750523,0.970639,23.376195,-49.832304,-19.137397,45.961165,4.797815,...,4.206811,-1.026016,-5.032163,5.067553,3.584452,-2.271772,-4.157420,7.516485,-2.606250,337099
84444,1.276644,38.879935,78.754535,0.329782,-28.717963,-29.557247,-134.062481,-18.697579,16.081745,-1.127570,...,20.521706,7.838622,-8.679683,2.151457,0.169054,-26.959183,-5.272316,-6.282950,-0.506783,84444


In [18]:
# Questions 2 tfidf weighted word2vec
df3_q2.head()

,0,1,2,3,4,5,6,7,8,9,...,375,376,377,378,379,380,381,382,383,id
181139,8.820910,40.525881,11.321742,21.863635,20.435016,-60.572040,-79.210767,-31.995541,1.458235,34.054208,...,21.261809,3.462853,-4.020733,8.553592,1.511645,-12.162342,0.292777,1.293257,2.729266,181139
284223,81.363026,20.975723,-2.690397,118.161789,0.586834,-23.837296,-105.329407,1.752800,-11.910041,43.749701,...,4.612265,1.066018,-13.171110,2.367111,12.641256,-5.212804,0.578862,15.698316,6.656775,284223
314840,36.034427,31.253479,57.069623,102.603941,-42.120048,14.044775,-71.787592,-29.243279,83.745689,32.965595,...,11.005654,2.224033,3.566263,1.024540,-0.785283,2.170033,2.167980,12.751685,6.469793,314840
337099,-9.196960,27.541945,30.818451,-3.334287,-8.577156,20.736038,-54.322146,-13.467081,47.400977,17.735614,...,5.063608,-0.598420,-5.403253,4.607515,2.329931,-2.605466,-5.248455,7.298447,-1.765915,337099
84444,7.502114,26.793544,47.562378,-10.846008,-21.162971,15.747988,-139.781573,12.793663,8.605174,-7.837080,...,26.102169,10.358936,-5.047877,0.962780,3.046357,-15.513790,1.539562,-2.585931,5.729592,84444


In [19]:
print("Number of features in nlp dataframe :", df1.shape[1])
print("Number of features in preprocessed dataframe :", df2.shape[1])
print("Number of features in question1 w2v  dataframe :", df3_q1.shape[1])
print("Number of features in question2 w2v  dataframe :", df3_q2.shape[1])
print("Number of features in final dataframe  :", df1.shape[1]+df2.shape[1]+df3_q1.shape[1]+df3_q2.shape[1])

Number of features in nlp dataframe : 29
Number of features in preprocessed dataframe : 769
Number of features in question1 w2v  dataframe : 385
Number of features in question2 w2v  dataframe : 385
Number of features in final dataframe  : 1568


In [40]:
# storing the final features to csv file
if not os.path.isfile('Desktop/finalfeaturestestin.csv'):
    df3_q1['id']=df1['id']
    df3_q2['id']=df1['id']
    df1  = df1.merge(df2, on='id',how='left')
    df2  = df3_q1.merge(df3_q2, on='id',how='left')
    result  = df1.merge(df2, on='id',how='left')
    result.to_csv('Desktop/finalfeaturestesting.csv')

In [1]:
import pandas as pd
import pickle
result=pd.read_csv('Desktop/finalfeatures2.csv')

In [41]:
pickle_out=open('dftesting','wb')
pickle.dump(result,pickle_out)

In [29]:
pickle_out.close()

In [31]:
del result['Unnamed: 0']